In [1]:
from voice_agent_flow.agents import SingleAgentRunner
from voice_agent_flow.llms import create_pydantic_azure_openai
from voice_agent_flow.agents import pmsg
from voice_agent_flow.agents.events import AgentTextStream

**正常定义一个agent和一个Runner，你需要的只是一个Runner而已**

In [2]:
from pydantic import BaseModel
from pydantic_ai import Agent
import time

class Person(BaseModel):
    name:str
    age:int
    
    def transfer(self):
        print("\n⚾️ I want to transfer to the next step.")
    
    
def weather_tool(location: str, credential = "3234980988908333498") -> str:
    """Search the weather for a location."""
    return f"The weather in {location} is snowing, the temperature will be below -5 degrees Celsius."

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(
    model, 
    tools = [weather_tool],
    instructions = (
    "You are a helpful assistant. you chat with users friendly."
    "When user asks questions about weather, do call the weather tool to get the weather information, and give suggestions on clothing based on the weather. "
    "When user mentions any person with name and age, you extract the name and age, return a json object defined by `Person` schema."
    "Reply in Chinese."
    ""),
    output_type = str | Person)

runner = SingleAgentRunner(agent)

**文本输出能够满足voice agent需求**

In [3]:
message_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather today in Beijing?"},
]

message_history = pmsg.to_history(message_history)

async for event in runner.run(message_history = message_history):
    if isinstance(event.event,AgentTextStream):
        time.sleep(0.05)
        print(event.event.delta, end = "")
    else:
        print(f"Received event: {event.event}")

Received event: ToolCallsOutputStart(status=None, message={'tool_name': 'weather_tool', 'args': '', 'tool_call_id': 'call_G73YgJZZoRPQlo05r1jglkt7', 'id': None, 'provider_details': None, 'part_kind': 'tool-call'})
Received event: ToolCallsOutput(status=None, message={'tool_name': 'weather_tool', 'args': '{"location":"北京市"}', 'tool_call_id': 'call_G73YgJZZoRPQlo05r1jglkt7', 'id': None, 'provider_details': None, 'part_kind': 'tool-call'})
Received event: ToolCallResult(status=None, message={'tool_name': 'weather_tool', 'content': 'The weather in 北京市 is snowing, the temperature will be below -5 degrees Celsius.', 'tool_call_id': 'call_G73YgJZZoRPQlo05r1jglkt7', 'metadata': None, 'timestamp': '2026-02-21T12:10:30.762074Z', 'part_kind': 'tool-return'})
今天北京市正在下雪，气温低于-5℃，非常寒冷。建议您穿厚羽绒服、帽子、手套和围巾，注意保暖。如果外出，请注意雪天路滑，安全第一！

**结构化输出能够满足voice agent需求**

In [4]:
message_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather today in Beijing?"},
    {"role": "assistant", "tool_name": "search", "args": "weather today", "tool_call_id": "1"},
    {"role": "tool", "tool_name": "search", "content": "The weather today is sunny with a high of 25°C.", "tool_call_id": "1"},
    {"role": "assistant", "content": "The weather today is sunny with a high of 25°C."},
    {'role': "user", "content": "Let me introduce my self, My name is Wang Huan and I am 32 years old, a middle age AI engineer living in the city of Beijing."}
]

message_history = pmsg.to_history(message_history)

async for event in runner.run(message_history = message_history):
    
    if isinstance(event.event,AgentTextStream):
        time.sleep(0.05)
        print(event.event.delta, end = "")
    else:
        print(f"Received event: {event.event}")

Received event: ToolCallsOutputStart(status=None, message={'tool_name': 'final_result', 'args': '', 'tool_call_id': 'call_7Tt5JkteJlX5qRFjlrnHdRcv', 'id': None, 'provider_details': None, 'part_kind': 'tool-call'})
Received event: StructuredOutput(status=None, message=Person(name='Wang Huan', age=32))
